In [ ]:
# usaremos o filtro 'warning' para deixar mais limpo.
import warnings
warnings.filterwarnings('ignore')

In [1]:
tickers = ['AMZN','AAPL','WBA','NOC','BA','LMT',
           'MCD','INTC','IBM','TXN','MA','MSFT',
           'GE','AXP','PEP','KO','JNJ','TM','HMC',
           'XOM','CVX','VLO','META','BAC']

In [2]:
start_date = '2010-01-01' 
end_date = '2020-11-01'

import yfinance as yf
df = yf.download(tickers, start_date, end_date)[['Adj Close']]

[*********************100%***********************]  27 of 27 completed

3 Failed downloads:
- NAV: No timezone found, symbol may be delisted
- SNE: No timezone found, symbol may be delisted
- MSBHY: No timezone found, symbol may be delisted


In [3]:
df

Adj Close                                                \
                  AAPL        AMZN        AXP          BA        BAC   
Date                                                                   
2010-01-04    6.496295    6.695000  33.437004   43.777546  12.977035   
2010-01-05    6.507525    6.734500  33.363468   45.211334  13.398851   
2010-01-06    6.404016    6.612500  33.902771   46.582809  13.555994   
2010-01-07    6.392177    6.500000  34.452625   48.468559  14.002625   
2010-01-08    6.434673    6.676000  34.428001   48.001022  13.878564   
...                ...         ...        ...         ...        ...   
2020-10-26  113.159950  160.352005  93.542404  160.830002  23.048315   
2020-10-27  114.684486  164.316498  91.659592  155.240005  22.400255   
2020-10-28  109.373199  158.139008  88.444305  148.139999  22.043356   
2020-10-29  113.425507  160.550507  87.932564  148.289993  22.109098   
2020-10-30  107.071640  151.807495  88.096718  144.389999  22.259377   

                                                                   ...  \
                  CVX         GE        HMC        IBM       INTC  ...   
Date                                                               ...   
2010-01-04  46.397545  68.084297  27.475611  79.247475  13.745538  ...   
2010-01-05  46.726177  68.436836  27.285515  78.290138  13.738959  ...   
2010-01-06  46.732052  68.084297  27.047918  77.781586  13.692875  ...   
2010-01-07  46.556007  71.609711  26.588531  77.512367  13.561213  ...   
2010-01-08  46.638153  73.152077  27.198399  78.290138  13.712626  ...   
...               ...        ...        ...        ...        ...  ...   
2020-10-26  62.941540  45.628902  23.034983  93.489319  42.632385  ...   
2020-10-27  61.672775  43.897732  22.714792  92.106377  41.646870  ...   
2020-10-28  59.339294  45.876217  21.904896  88.848999  40.378479  ...   
2020-10-29  61.042831  45.567085  22.422850  90.731773  40.250729  ...   
2020-10-30  61.663902  45.876217  22.243921  93.022774  40.405857  ...   

                                                                    \
                  MSFT NAV         NOC         PEP SNE          TM   
Date                                                                 
2010-01-04   23.572371 NaN   38.601650   41.343437 NaN   72.314590   
2010-01-05   23.579985 NaN   38.669563   41.843014 NaN   71.201126   
2010-01-06   23.435278 NaN   38.818924   41.424431 NaN   72.110580   
2010-01-07   23.191549 NaN   39.219551   41.161148 NaN   71.218124   
2010-01-08   23.351498 NaN   39.002277   41.026138 NaN   72.892555   
...                ...  ..         ...         ...  ..         ...   
2020-10-26  204.898468 NaN  296.256622  127.956718 NaN  132.850006   
2020-10-27  207.990280 NaN  288.875092  128.939163 NaN  132.460007   
2020-10-28  197.680984 NaN  280.786011  124.351395 NaN  130.070007   
2020-10-29  199.670685 NaN  279.571716  124.184570 NaN  131.350006   
2020-10-30  197.476181 NaN  277.114380  123.535789 NaN  131.289993   

                                                         
                   TXN        VLO        WBA        XOM  
Date                                                     
2010-01-04   18.363136  10.440829  25.593069  41.319061  
2010-01-05   18.257242  10.709294  25.387226  41.480389  
2010-01-06   18.123112  10.977755  25.195099  41.838905  
2010-01-07   18.179585  11.059456  25.346056  41.707455  
2010-01-08   18.596119  10.907721  25.380363  41.540157  
...                ...        ...        ...        ...  
2020-10-26  134.720016  34.642689  32.911259  29.030132  
2020-10-27  135.799942  33.521221  31.841043  28.568781  
2020-10-28  131.996964  31.707613  30.593933  27.480694  
2020-10-29  135.910965  32.881638  29.647551  28.699352  
2020-10-30  134.423462  33.827873  30.107473  28.394686  

[2727 rows x 27 columns]

In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

### End.